In [1]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920,1080))
display.start()

In [3]:
webdriver.ChromeOptions()

In [4]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# tox-to-reproduction-header
tox_to_reproduct_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity-header"]/div/div[2]'
driver.find_element_by_xpath(tox_to_reproduct_path).click()
time.sleep(0.5)

# header 하위 3개의 창
tox_to_repro_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity"]/div/div/div[1]/div[3]/button'
driver.find_element_by_xpath(tox_to_repro_path).click()
time.sleep(0.5)

/tmp/ipykernel_3890/1650006056.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [5]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG408경로, 이건 바꿔야함
TG_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[24]'
driver.find_element_by_xpath(TG_path).click()
time.sleep(0.5)

#EP870.3100경로, 이건 바꿔야함
EP_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[6]'
driver.find_element_by_xpath(EP_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [6]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [7]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(0.5)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg408_source.json', 'w'))

100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.12s/it, page=704]
10838.495966434479


## json전처리

In [51]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg408_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []
err_count=0
for i in tqdm(range(len(df))):
#for i in tqdm(range(10)):
    CE = np.nan
    EL = np.nan
    RH = np.nan
    CS = np.nan
    MT = np.nan
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        # Critical effects observed:
        admin_tmp = soup.find_all("dl", {"class" : "HorDL"})

        for j in range(len(admin_tmp)):
            if "<dt>Critical effects observed:</dt>" in str(admin_tmp[j]):
                CE = str(admin_tmp[j]).split('<dt>Critical effects observed:</dt><dd>')[1]
                CE = CE.split('</dd>')[0]
            chem_dict['Critical effects'] = CE
            
        # Basis for effect level:
        for j in range(len(admin_tmp)):
            if "<dt>Basis for effect level:</dt>" in str(admin_tmp[j]):
                EL = str(admin_tmp[j]).split('<dt>Basis for effect level:</dt><dd>')[1]
                EL = EL.split('</dd>')[0]
            chem_dict['effect level'] = EL
            
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sMaterialsAndMethods'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Route of administration:')
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Route of administration'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Route of administration'] = np.nan

            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        err_count += 1
        pass
    

print(f'error count : {err_count}')
# %%
result = pd.DataFrame(result_)

100%|███████████████████████████████████████████████████████████████████████████████| 3517/3517 [04:00<00:00, 14.63it/s]

error count : 117


In [61]:
result['Route of administration'].value_counts()

oral: gavage                                                                    3612
oral: feed                                                                      1171
oral: drinking water                                                             138
oral: unspecified                                                                108
oral: capsule                                                                     15
other: 2. Gavage; 3. Feed                                                          4
other: Intraveinous                                                                4
other: mixed with liquid feed                                                      4
other: Oral: syringe and a No. 16 stainless steel ball-tipped feeding needle       2
other: both feed and water                                                         2
other: oral: diet                                                                  2
other:                                                           

In [52]:
result['Dose descriptor'].value_counts()

NOAEL                                                                     3503
NOEL                                                                       995
LOAEL                                                                      454
LOEL                                                                        53
dose level:                                                                 11
other: based on the results, doses were selected for OECD TG 421 study       7
other:                                                                       6
other: No NOEL identified                                                    3
BMDL10                                                                       3
other: maximum tolerated dose                                                2
other: selection of dose levels for subsequent 28-day toxicity study         2
other: NOEL and NOAEL                                                        2
other: NOTEL                                        

## value 전처리

In [65]:
result_NOAELS = result[result['Dose descriptor'].isin(['NOAEL', 'NOEL'])].reset_index(drop = True)
result_NOAELS['Dose descriptor'].unique()

array(['NOAEL', 'NOEL'], dtype=object)

In [66]:
result['Effect level'].value_counts()

1 000  mg/kg bw/day (nominal)                    503
1 000  mg/kg bw/day (actual dose received)       319
100  mg/kg bw/day (nominal)                      205
>= 1 000  mg/kg bw/day (actual dose received)    143
100  mg/kg bw/day (actual dose received)         117
                                                ... 
845  mg/kg bw/day (nominal)                        1
704  mg/kg bw/day (nominal)                        1
690  mg/kg bw/day (nominal)                        1
> 90  mg/kg bw/day (actual dose received)          1
>= 18 700  mg/kg bw/day (nominal)                  1
Name: Effect level, Length: 881, dtype: int64

In [70]:
# Value에 other: ~~ 이렇게 돼있는거 제거
other_idx = [i for i in range(len(result_NOAELS)) if 'other' in str(result_NOAELS['Effect level'][i])]
result_NOAELS['Effect level'][other_idx] = [re.sub(' other: ', '', result_NOAELS['Effect level'][i]) for i in other_idx]
result_NOAELS['Effect level'].value_counts()[0:20]

1 000  mg/kg bw/day (nominal)                    510
1 000  mg/kg bw/day (actual dose received)       313
100  mg/kg bw/day (nominal)                      208
>= 1 000  mg/kg bw/day (actual dose received)    143
50  mg/kg bw/day (nominal)                       103
100  mg/kg bw/day (actual dose received)          96
150  mg/kg bw/day (nominal)                       95
300  mg/kg bw/day (actual dose received)          92
200  mg/kg bw/day (nominal)                       89
50  mg/kg bw/day (actual dose received)           87
200  mg/kg bw/day (actual dose received)          82
300  mg/kg bw/day (nominal)                       81
150  mg/kg bw/day (actual dose received)          71
>= 1 000  mg/kg bw/day (nominal)                  70
10  mg/kg bw/day (nominal)                        65
15  mg/kg bw/day (nominal)                        56
500  mg/kg bw/day (actual dose received)          49
10  mg/kg bw/day (actual dose received)           47
15  mg/kg bw/day (actual dose received)       

In [71]:
# Value에 ca. ~ 이렇게 돼있는거에서 ca. 제거
ca_idx = [i for i in range(len(result_NOAELS)) if 'ca.' in str(result_NOAELS['Effect level'][i])]
result_NOAELS['Effect level'][ca_idx] = [re.sub('ca\. ', '', result_NOAELS['Effect level'][i]) for i in ca_idx]
result_NOAELS['Effect level'].value_counts()[0:20]

1 000  mg/kg bw/day (nominal)                    510
1 000  mg/kg bw/day (actual dose received)       313
100  mg/kg bw/day (nominal)                      208
>= 1 000  mg/kg bw/day (actual dose received)    143
50  mg/kg bw/day (nominal)                       103
100  mg/kg bw/day (actual dose received)          96
150  mg/kg bw/day (nominal)                       95
300  mg/kg bw/day (actual dose received)          92
200  mg/kg bw/day (nominal)                       89
50  mg/kg bw/day (actual dose received)           87
200  mg/kg bw/day (actual dose received)          82
300  mg/kg bw/day (nominal)                       81
150  mg/kg bw/day (actual dose received)          71
>= 1 000  mg/kg bw/day (nominal)                  70
10  mg/kg bw/day (nominal)                        65
15  mg/kg bw/day (nominal)                        56
500  mg/kg bw/day (actual dose received)          49
10  mg/kg bw/day (actual dose received)           47
15  mg/kg bw/day (actual dose received)       

In [75]:
np.unique([re.findall('\(.*?\)', str(i)) for i in result_NOAELS['Effect level']])
result_NOAELS['Effect level'] = [re.sub('\(.*?\)', '', str(i)) for i in result_NOAELS['Effect level']]
result_NOAELS['Effect level'].value_counts()[0:20]

1 000  mg/kg bw/day        823
100  mg/kg bw/day          304
>= 1 000  mg/kg bw/day     213
50  mg/kg bw/day           190
300  mg/kg bw/day          173
200  mg/kg bw/day          171
150  mg/kg bw/day          166
10  mg/kg bw/day           112
15  mg/kg bw/day            95
500  mg/kg bw/day           76
30  mg/kg bw/day            69
> 1 000  mg/kg bw/day       59
20  mg/kg bw/day            56
250  mg/kg bw/day           55
25  mg/kg bw/day            50
5  mg/kg bw/day             49
40  mg/kg bw/day            45
nan                         44
125  mg/kg bw/day           36
600  mg/kg bw/day           28
Name: Effect level, dtype: int64

In [80]:
val_df = result_NOAELS.copy()
val_df['Value_split'] = [val_df['Effect level'][i].split(' ') for i in range(len(val_df))]

In [82]:
range_idx = [i for i in range(len(val_df)) if '-' in str(result_NOAELS['Effect level'][i])]
idx = set(list(range(val_df.shape[0]))) - set(range_idx)

In [83]:
def isFloat(string):
    try:
        float(string)
        return True
    
    except ValueError:
        return False

In [84]:
val_df['lower_ineq'] = ''

for i in tqdm(idx):
    if isFloat(val_df['Value_split'][i][0]):
        val_df['lower_ineq'][i] = np.nan
        
    else:
        val_df['lower_ineq'][i] = val_df['Value_split'][i][0]
        val_df['Value_split'][i].remove(val_df['Value_split'][i][0])


100%|████████████████████████████████████████████████████████████████████████████| 4468/4468 [00:00<00:00, 11645.05it/s]


In [85]:
val_df['lower_value'] = ''

for i in tqdm(idx):
    if isFloat(''.join(val_df['Value_split'][i][:4])):
        val_df['lower_value'][i] = float(''.join(val_df['Value_split'][i][:4]))
        del val_df['Value_split'][i][:4]
        
    elif isFloat(''.join(val_df['Value_split'][i][:3])):
        val_df['lower_value'][i] = float(''.join(val_df['Value_split'][i][:3]))
        del val_df['Value_split'][i][:3]
    
    elif isFloat(''.join(val_df['Value_split'][i][:2])):
        val_df['lower_value'][i] = float(''.join(val_df['Value_split'][i][:2]))
        del val_df['Value_split'][i][:2]
    
    elif isFloat(''.join(val_df['Value_split'][i][:1])):
        val_df['lower_value'][i] = float(''.join(val_df['Value_split'][i][:1]))
        del val_df['Value_split'][i][:1]

100%|████████████████████████████████████████████████████████████████████████████| 4468/4468 [00:00<00:00, 10597.24it/s]


In [86]:
# 단위
# val_df['unit'] = ''
u_ = ['mg/kg', 'ml', 'ppm']
val_df['unit'] = val_df['Effect level'].apply(lambda x: ''.join(y for y in x.split() if y in u_))

for i in idx:
    try:
        val_df['Value_split'][i].remove(val_df['unit'][i])
    except ValueError:
        pass

In [88]:
val_df['upper_ineq'] = ''
for i in range_idx:
    try:
        range_ineq = re.findall('>\=|<\=|>|<', val_df['Effect level'][i])
        val_df['lower_ineq'][i] = range_ineq[0]
        val_df['upper_ineq'][i] = range_ineq[1]
        
        val_df['Value_split'][i] = [j for j in val_df['Value_split'][i] if j not in range_ineq]
        
    except IndexError:
        val_df['lower_ineq'][i] = np.nan
        val_df['upper_ineq'][i] = np.nan

In [89]:
# value
val_df['upper_value'] = ''

for i in range_idx:
    try:
        if val_df['Value_split'][i].index('-') == 1:
            val_df['lower_value'][i] = float(val_df['Value_split'][i][0])
        # val_df['Value_split'][i].remove(val_df['Value_split'][i][0])
        
            if isFloat(''.join(val_df['Value_split'][i][2:4])):
                val_df['upper_value'][i] = float(''.join(val_df['Value_split'][i][2:4]))
                # val_df['Value_split'][i].remove(''.join(val_df['Value_split'][i][2:4]))
            else:
                val_df['upper_value'][i] = float(val_df['Value_split'][i][2])
                # val_df['Value_split'][i].remove(val_df['Value_split'][i][2])
            
        elif val_df['Value_split'][i].index('-') == 2:
            val_df['lower_value'][i] = float(''.join(val_df['Value_split'][i][:2]))
            val_df['upper_value'][i] = float(''.join(val_df['Value_split'][i][3:5]))
    
    except ValueError:
        val_df['lower_value'][i] = np.nan
        val_df['upper_value'][i] = np.nan
        

In [111]:
val_df.drop(['Value_split'], axis = 1, inplace = True)
val_df[10:30]

Chemical      CasRN Critical effects  \
10  2-methyl-2H-isothiazol-3-one  2682-20-4    not specified   
11   Diammonium peroxodisulphate  7727-54-0    not specified   
12   Diammonium peroxodisulphate  7727-54-0    not specified   
13   Diammonium peroxodisulphate  7727-54-0    not specified   
14   Diammonium peroxodisulphate  7727-54-0    not specified   
15   Diammonium peroxodisulphate  7727-54-0    not specified   
16   Diammonium peroxodisulphate  7727-54-0    not specified   
17   Diammonium peroxodisulphate  7727-54-0    not specified   
18   Diammonium peroxodisulphate  7727-54-0    not specified   
19           Triphenyl phosphate   115-86-6    not specified   
20           Triphenyl phosphate   115-86-6    not specified   
21           Triphenyl phosphate   115-86-6    not specified   
22           Triphenyl phosphate   115-86-6    not specified   
23                 2-furaldehyde    98-01-1    not specified   
24                 2-furaldehyde    98-01-1    not specified   
25  2-(phenylmethoxy)naphthalene   613-62-7    not specified   
26  2-(phenylmethoxy)naphthalene   613-62-7    not specified   
27  2-(phenylmethoxy)naphthalene   613-62-7    not specified   
28              3-methylpyrazole  1453-58-3              yes   
29              3-methylpyrazole  1453-58-3              yes   

                                         effect level Route of administration  \
10  other: Decreased cumulative body weight gain i...    oral: drinking water   
11                        body weight and weight gain              oral: feed   
12                        body weight and weight gain              oral: feed   
13                        body weight and weight gain              oral: feed   
14                        body weight and weight gain              oral: feed   
15                        body weight and weight gain              oral: feed   
16                        body weight and weight gain              oral: feed   
17                        body weight and weight gain              oral: feed   
18                        body weight and weight gain              oral: feed   
19  other: liver weight in crease in females of th...              oral: feed   
20  other: liver weight in crease in females of th...              oral: feed   
21  other: liver weight in crease in females of th...              oral: feed   
22  other: liver weight in crease in females of th...              oral: feed   
23  other: haematological and microscopic liver ch...              oral: feed   
24  other: haematological and microscopic liver ch...              oral: feed   
25               other: recalculation from mg/kg diet              oral: feed   
26               other: recalculation from mg/kg diet              oral: feed   
27               other: recalculation from mg/kg diet              oral: feed   
28  other: toxicological endpoint: effects on Clar...    oral: drinking water   
29  other: toxicological endpoint: effects on Clar...    oral: drinking water   

   Dose descriptor         Effect level  \
10           NOAEL  24.6  mg/kg bw/day    
11           NOAEL  24.6  mg/kg bw/day    
12           NOAEL  24.6  mg/kg bw/day    
13           NOAEL  24.6  mg/kg bw/day    
14           NOAEL    91  mg/kg bw/day    
15           NOAEL    91  mg/kg bw/day    
16           NOAEL    91  mg/kg bw/day    
17           NOAEL    91  mg/kg bw/day    
18           NOAEL    91  mg/kg bw/day    
19           NOAEL   105  mg/kg bw/day    
20           NOAEL   117  mg/kg bw/day    
21           NOAEL   105  mg/kg bw/day    
22           NOAEL   117  mg/kg bw/day    
23           NOAEL    53  mg/kg bw/day    
24           NOAEL    53  mg/kg bw/day    
25           NOAEL    1 000  mg/kg diet   
26           NOAEL    98  mg/kg bw/day    
27           NOAEL    82  mg/kg bw/day    
28           NOAEL     5  mg/kg bw/day    
29           NOAEL     5  mg/kg bw/day    

                                   Based on          Sex  \
10           

In [114]:
val_df.astype({'lower_value':'float64'})

ValueError: could not convert string to float: ''

In [115]:
# conclude value
for i in range(len(val_df)):
    try : 
        if val_df['lower_value'][i] > 100:
            target = 0
        elif val_df['lower_value'][i] > 10:
            target = 1
        elif val_df['lower_value'][i] > 0:
            target = 2
        else : target = np.nan
    except TypeError :
        target = 0
    val_df.loc[i,'class'] = target
val_df['class'].value_counts()

0.0    2847
1.0    1199
2.0     391
Name: class, dtype: int64

In [116]:
# save df
val_df.to_csv('tg408_pre1.csv', header = True, index = False)

# smiles 추가

In [1]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('tg408_pre1.csv')
len(data['CasRN'].unique())

1961

In [3]:
data.head()

Chemical        CasRN  \
0             Tris(2-chloro-1-methylethyl) phosphate   13674-84-5   
1                                          Triclosan    3380-34-5   
2  3-ethoxy-1,1,1,2,3,4,4,5,5,6,6,6-dodecafluoro-...  297730-93-9   
3  3,3,4,4,5,5,6,6,7,7,8,8,8-tridecafluorooctyl a...   17527-29-6   
4  3,3,4,4,5,5,6,6,7,7,8,8,8-tridecafluorooctyl a...   17527-29-6   

  Critical effects                                       effect level  \
0    not specified  other: the increase in absolute and relative l...   
1    not specified  other: (corresponding to ca. 107 and 113 mg/kg...   
2               no                     histopathology: non-neoplastic   
3    not specified  other: Differences in clinical chemistry and h...   
4    not specified  other: Differences in clinical chemistry and h...   

  Route of administration Dose descriptor          Effect level    Based on  \
0              oral: feed           NOAEL            2 500  ppm         NaN   
1              oral: feed           NOAEL            1 000  ppm         NaN   
2            oral: gavage           NOAEL  1 000  mg/kg bw/day   act. ingr.   
3            oral: gavage           NOAEL     25  mg/kg bw/day    test mat.   
4            oral: gavage           NOAEL      5  mg/kg bw/day    test mat.   

           Sex                             Basis for effect level  \
0       female                                                NaN   
1  male/female  other: (corresponding to ca. 33.5 and 34 mg/kg...   
2  male/female  histopathology: non-neoplastic. organ weights ...   
3         male                                other: see 'Remark'   
4       female  other: Differences in clinical chemistry and h...   

               Remarks Remarks on result  Critical effects observed  \
0                  NaN               NaN                        NaN   
1                  NaN               NaN                        NaN   
2                  NaN               NaN                        NaN   
3  (systemic toxicity)               NaN                        NaN   
4  (systemic toxicity)               NaN                        NaN   

  lower_ineq  lower_value   unit upper_ineq  upper_value  value  class  
0        NaN       2500.0    ppm        NaN          NaN    NaN    0.0  
1        NaN       1000.0    ppm        NaN          NaN    NaN    0.0  
2        NaN       1000.0  mg/kg        NaN          NaN    NaN    0.0  
3        NaN         25.0  mg/kg        NaN          NaN    NaN    1.0  
4        NaN          5.0  mg/kg        NaN          NaN    NaN    2.0

In [4]:
data['class'].unique()
data['class'].isna().sum()

data = data[data['class'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['class'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [5]:
data = data.drop_duplicates()
len(data)

2648

In [6]:
data['class'].value_counts()

0.0    1667
1.0     748
2.0     233
Name: class, dtype: int64

In [7]:
data1 = data[:1400]
data2 = data[1400:]

In [8]:
tqdm.pandas()
data1['SMILES'] = data1.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|███████████████████████████████████████████████████████████████████████████████| 1400/1400 [24:55<00:00,  1.07s/it]


In [9]:
tqdm.pandas()
data2['SMILES'] = data2.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|███████████████████████████████████████████████████████████████████████████████| 1248/1248 [22:19<00:00,  1.07s/it]


In [10]:
data = pd.concat([data1, data2])
data = data[['Chemical','CasRN','SMILES','Dose descriptor','lower_value','class']]

In [13]:
data.to_csv('tg408_2smiles.csv', header = True, index = False)

In [15]:
data.SMILES.isna().sum()
data = data[data['SMILES'].notna()].reset_index(drop = True)

In [16]:
data.to_csv('tg408_3deletenull.csv', header = True, index = False)

In [ ]:
## 